Import di base

In [1]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from IPython.display import clear_output

import numpy as np
import random
import socket
import time

import struct



In [2]:
# definizioni variabili globali ==================================================================
HOST = "127.0.0.1"
PORT = 7001
PACKET_SIZE = 1024

PacketCounter = 0
PacketCounter_lastCheck = 0

# definizioni funzioni =======================================================

#creo socket e lo faccio partire
def CreaSocket():
    # creo socket server per UE5
    MySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    #Bindo il socket 
    MySocket.bind((HOST,PORT))

    #accendo il socket in ascolto
    MySocket.listen()

    return MySocket


#ciclo di attesa connesione
def AttesaConnesione(MySocket):
    while True:
        RemoteConnection , RemoteAddress = MySocket.accept()
        MySocket.close()
        break
    return RemoteConnection,RemoteAddress

#funzione per sampolare il robot di unreal
def MandaRicevi_Robot(MySocket, R_Start=0.0, R_Arm_1=0.0, R_Arm_2=0.0, R_Arm_3=0.0, R_Arm_4=0.0, R_End=0.0):
    MySocket.sendall(bytes(
                    str(R_Start) +"|"+ 
                    str(R_Arm_1) +"|"+ 
                    str(R_Arm_2) +"|"+ 
                    str(R_Arm_3) +"|"+ 
                    str(R_Arm_4) +"|"+ 
                    str(R_End)
                    ,"utf-8"))
    
    ReceivedData = MySocket.recv(PACKET_SIZE)
    # ricevi in stringa //ReceivedData = ReceivedData.decode("utf-8")
    print(ReceivedData)
    print(struct.unpack("f",ReceivedData))
    return float(ReceivedData)

# classe environment di gioco =============================================

class RobotEnv(Env):
    def __init__(self):
        # azioni che possiamo fare
        self.action_space = Discrete(6)
        # Array distanze dal target
        self.observation_space = Box(low=-10000,high=10000,shape=(3,))
        # distanza di partenza
        self.state = [0.0,0.0,0.0]

        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)

        # lunghezza esercizio
        self.Exercise_length = 800

    def step(self, action,verbosita=False):
        global UE5_Socket
        
        # applichiamo azione
        
        if action == 0:
            self.angoloAttuale_1 += 2
        elif action == 1:
            self.angoloAttuale_1 -= 2
        elif action == 2:
            pass

        if action == 3:
            self.angoloAttuale_2 += 2
        elif action == 4:
            self.angoloAttuale_2 -= 2
        elif action == 5:
            pass

        self.state[2] = self.angoloAttuale_2
        self.state[1] = self.angoloAttuale_1
        self.state[0] = MandaRicevi_Robot(UE5_Socket, -90.0, self.angoloAttuale_1, self.angoloAttuale_2)

        # riduzione tempo 
        self.Exercise_length -= 1

        #calcolo reward
        if self.state[0] <= 40:
            reward = 1
        else:
            reward = -3 * (self.state[0] / 100)

        done = False

        #verfica se tempo finito
        if self.Exercise_length <= 0:
            done = True
        else:
            done = False
               
        # segnaposto per info
        info = {}
        
        # ritorniamo informazioin sullo step
        return self.state, reward, done, info

    def render(self,mode=0):
        print("Tempo: "+str(self.Exercise_length) + "-- distanza: " + str(self.state))

    def reset(self):
        #resettiamo temperatura
        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)
        self.state = [0.0,0.0,0.0]
        # resettiamo tempo
        self.Exercise_length = 800
        return self.state

# codice ==================================================================

#creo e ascolto
Server_Socket = CreaSocket()

#attendo e connetto con UE5 
UE5_Socket , UE5_Address = AttesaConnesione(Server_Socket)

#confermo connesione aperta
print(f"Aperto socket con UE5: {UE5_Address}")


SimpleRobotEnv = RobotEnv()


Aperto socket con UE5: ('127.0.0.1', 56288)


In [6]:
# giro di prova dell'exercise
episodes = 1
for episode in range(1, episodes+1):
    state = SimpleRobotEnv.reset()
    done = False
    score = 0 
    
    while not done:
        SimpleRobotEnv.render(mode=0)
        action = SimpleRobotEnv.action_space.sample()
        n_state, reward, done, info = SimpleRobotEnv.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Tempo: 800-- distanza: [0.0, 0.0, 0.0]


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow
tensorflow.device("cpu:0")

states = SimpleRobotEnv.observation_space.shape
actions = SimpleRobotEnv.action_space.n

def build_model(states,actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + SimpleRobotEnv.observation_space.shape))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(actions,activation="linear"))
    return model

In [58]:
del model # da usare quando da errore sotto

In [59]:
model = build_model(states,actions)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                256       
_________________________________________________________________
dense_21 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 390       
Total params: 8,966
Trainable params: 8,966
Non-trainable params: 0
_________________________________________________________________


In [60]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannGumbelQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [61]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=["mae"])
dqn.fit(SimpleRobotEnv, nb_steps=500000,visualize=False,verbose=1)

Training for 500000 steps ...
Interval 1 (0 steps performed)
 9718/10000 [============================>.] - ETA: 12s - reward: -3.3213done, took 418.277 seconds


In [84]:
scores = dqn.test(SimpleRobotEnv, nb_episodes=10,visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 10 episodes ...
Episode 1: reward: 896.452, steps: 1000
Episode 2: reward: 985.179, steps: 1000
Episode 3: reward: 1000.000, steps: 1000
Episode 4: reward: 972.865, steps: 1000
Episode 5: reward: 637.557, steps: 1000
Episode 6: reward: 966.683, steps: 1000


KeyboardInterrupt: 

Chiusure varie

In [ ]:
# spengo socket prima di uscire
UE5_Socket.shutdown(socket.SHUT_RDWR)
UE5_Socket.close()

Codice vario utile:

In [ ]:
'''
DistanzaAttuale = MandaRicevi_Robot(
                            UE5_Socket,
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0)
                            )
'''

'\nDistanzaAttuale = MandaRicevi_Robot(\n                            UE5_Socket,\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0)\n                            )\n'